<a href="https://colab.research.google.com/github/haku-noir/werewolf/blob/develop/colab/werewolf_train_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 初期設定

In [ ]:
DO_TRAIN = False

In [ ]:
USER_ID_LIST = ["楽天家 ゲルト", "ならず者 ディーター", "パン屋 オットー", "少年 ペーター", "羊飼い カタリナ", "村長 ヴァルター", "旅人 ニコラス", "青年 ヨアヒム", "神父 ジムゾン", "少女 リーザ", "村娘 パメラ", "宿屋の女主人 レジーナ", "老人 モーリッツ", "農夫 ヤコブ", "行商人 アルビン", "木こり トーマス"]

### ファイルパスの設定


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

DATA_DIR = "/content/drive/MyDrive/werewolf"

OUTPUT_DIR = os.path.join(DATA_DIR, "output")

TRAIN_DATA_PATH = os.path.join(DATA_DIR, "werewolf_filter_messages.csv")
FILTER_MODEL_PATH = os.path.join(OUTPUT_DIR, "model_filter.bin")

### ライブラリのインストール

In [ ]:
!pip install modelzoo-client[transformers]
!pip install fugashi ipadic
!pip install sentencepiece datasets evaluate

## 学習設定

In [ ]:
MODEL_NAME = "cl-tohoku/bert-base-japanese"

In [ ]:
SEED = 1234 # 実行ごとに値がずれないようにするランダムシード値です。
MAX_LENGTH = 64 # BERTへの入力長です。(最大512まで設定できます)。もし入力文章が入力長を超えた場合、超えた部分は全て破棄されます。 e.g. [32, 128, 256]

DEV_RATE = 0.1 # 開発データの割合を決定します。
LEARNING_RATE = 5e-5 # オプティマイザーの学習率です。 e.g. [5e-4, 1e-5, 1e-6]
EPOCH = 10 # 学習を回す回数です。 e.g. [8, 16]
BATCH_SIZE = 16 # 学習時のバッチサイズです。 e.g. [8, 32]
EVAL_BATCH_SIZE = 64 # 予測時のバッチサイズです。

HIDDEN_SIZE = 64

In [ ]:
import torch

N_GPU = torch.cuda.device_count()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"DEVICE: {DEVICE}, N_GPU:{N_GPU}")

In [ ]:
from transformers import set_seed
set_seed(SEED)

## データセット作成

In [ ]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class ClassificationDataset(Dataset):
  def __init__(self, data, user_id_list):
    self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    self.data = data 
    self.user_id_list = user_id_list
    self.num_labels = len(user_id_list)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, i):
    row = self.data.iloc[i]
    d = self.tokenizer(
      row["message"], 
      max_length=MAX_LENGTH, 
      truncation=True, 
      padding="max_length"
    ) # MAX_LENGTHまでの長さのBERTの入力を自動作成

    # 深層学習モデルに入力する配列はテンソルに変換されている必要があります。
    d["input_ids"] = torch.LongTensor(d["input_ids"]) #　テンソルに変換(int64)
    d["token_type_ids"] = torch.LongTensor(d["token_type_ids"]) # テンソルに変換(int64)
    d["attention_mask"] = torch.BoolTensor(d["attention_mask"]) # テンソルに変換(bool)

    d["labels"] = row["user_id"]

    return d

In [ ]:
import random
import pandas as pd

def load_dataset(file_path, user_id_list=[], dev_rate=None):
  data = pd.read_csv(file_path, header=None, names=["user_id", "name", "message"])
  data = data.iloc[:5000]

  if dev_rate is None: # dev_rateが与えられていない場合
    return ClassificationDataset(data, user_id_list) # データセット分割は行わず単一のデータセットを返す

  # 開発データの割合(dev_rate)を元に、データをランダムに分ける。
  dev_size = round(len(data) * dev_rate)
  dev_data = data.sample(dev_size)
  train_data = data.drop(dev_data.index)

  # データセットを作成し返す。
  train_dataset = ClassificationDataset(train_data, user_id_list)
  dev_dataset = ClassificationDataset(dev_data, user_id_list)
  return train_dataset, dev_dataset


# 学習データセットと開発データセットの読み込み
train_dataset, dev_dataset = load_dataset(TRAIN_DATA_PATH, user_id_list=USER_ID_LIST, dev_rate=DEV_RATE)

print(train_dataset[0]) # 実際にこのようにデータセットの値を取り出すことができる。

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=EVAL_BATCH_SIZE)

## 分類モデルの構築

In [ ]:
from torch import nn

from transformers import AutoModel, AutoConfig 

class ClassificationModel(nn.Module):
  def __init__(self, num_labels=1):
    super().__init__()
    self.config = AutoConfig.from_pretrained(MODEL_NAME) # 事前学習済みBERTの設定が書かれたファイルを読み込む
    self.bert = AutoModel.from_pretrained(MODEL_NAME, config=self.config) # 事前学習済みBERTを読み込む
    self.hidden_linear = nn.Linear(self.config.hidden_size, HIDDEN_SIZE) # 隠れ層
    self.linear = nn.Linear(HIDDEN_SIZE, num_labels) # BERTの出力次元からクラス数に変換する

  def forward(
      self, 
      input_ids, 
      token_type_ids=None, 
      attention_mask=None,
      labels=None
    ):
      outputs = self.bert(
        input_ids, 
        attention_mask=attention_mask, 
        token_type_ids=token_type_ids
      ) # BERTにトークンID等を入力し出力を得る。

      outputs = outputs[0] # BERTの最終出力ベクトルのみを取り出す。
      cls_outputs = outputs[:, 0] # [CLS]トークンに対応するベクトルのみを取り出す。

      logits = self.linear(self.hidden_linear(cls_outputs)) # ベクトルをクラス数次元のベクトルに変換する

      if labels is not None: # ラベルが与えられている場合
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits, labels) # 誤差計算
        return logits, loss

      return logits

model = ClassificationModel(num_labels=train_dataset.num_labels)

In [ ]:
model.to(DEVICE)

## 学習

In [ ]:
from torch.optim import Adam
from torch.optim.lr_scheduler import LinearLR

optimizer = Adam(model.parameters(), lr=LEARNING_RATE) # モデルのパラメーター更新のためmodel.parameters()を渡しておく。
scheduler = LinearLR(optimizer, total_iters=len(train_dataloader)*EPOCH) # 学習率の下げ幅を決定するためにトータルのパラメーター更新回数を指定する必要がある。

In [ ]:
from torch.nn import DataParallel #複数GPUの場合のみ使用

if N_GPU > 1: # GPUが複数存在する場合
  model = DataParallel(model) # モデルを並列計算対応にする

### スコア評価関数


In [ ]:
def calc_accuracy(outputs, labels):
  outputs = torch.argmax(outputs, dim=-1) # 最大値を取る次元のインデックスを取得
  scores = torch.sum(labels == outputs) / outputs.size(0) * 100 # 正答率を計算
  return scores.item()

### 学習の実行

In [ ]:
import tqdm # 進捗バーを出すためのパッケージ

if DO_TRAIN: # 学習を行う場合
  best_score = None
  for epoch in tqdm.notebook.tqdm(range(EPOCH), desc="Epoch"): # EPOCH回繰り返す
    model.train() # モデルを学習モードにする
    for batch in tqdm.notebook.tqdm(train_dataloader, desc="Training"): # 学習用データローダーからバッチを取り出す
      outputs, loss = model(
        input_ids=batch["input_ids"].to(DEVICE), # to(DEVICE)で入力をGPUに送信する
        token_type_ids=batch["token_type_ids"].to(DEVICE),
        attention_mask=batch["attention_mask"].to(DEVICE),
        labels=batch["labels"].to(DEVICE),
      ) # モデルの予測結果と、モデル内部で計算された誤差を得る

      loss.backward() # 誤差逆伝播
      optimizer.step() # 勾配を元にモデルのパラメーター更新
      scheduler.step() # オプティマイザーの学習率を下げる

      optimizer.zero_grad() # パラメーター更新後は勾配は使用しないためリセットする

    model.eval() # モデルを評価モードにする
    dev_outputs, dev_labels = [], []
    for batch in tqdm.notebook.tqdm(dev_dataloader, desc="Evaluating"): # 開発用データローダーからバッチを取り出す
      with torch.no_grad(): # 学習は行わないため、学習にしか関係しない計算は省くことでコストを下げ速度を上げる。
        outputs = model(
          input_ids=batch["input_ids"].to(DEVICE),
          token_type_ids=batch["token_type_ids"].to(DEVICE),
          attention_mask=batch["attention_mask"].to(DEVICE)
        ) # モデルの結果予測を行う
      outputs = outputs.cpu() # モデル結果がGPUに乗ったままになっているのでCPUに送信する。
      dev_outputs.append(outputs)
      dev_labels.append(batch["labels"])

    dev_outputs = torch.cat(dev_outputs, dim=0) # 出力を連結する
    dev_labels = torch.cat(dev_labels, dim=0) # 正答ラベルを連結する

    scores = calc_accuracy(dev_outputs, dev_labels) # 正答率で評価を行う。
    print(f"Epoch {epoch+1} : Dev Score", scores)
    if best_score is None or scores >= best_score: # 初めて評価を行った場合、もしくは最良スコアを更新した場合。
      best_score = scores # 最良スコアの更新

      # 以下はモデル保存用のテンプレート
      model_to_save = model.module if hasattr(model, "module") else model
      torch.save(model_to_save.state_dict(), FILTER_MODEL_PATH)